In [33]:
import import_ipynb
import configparser
import numpy as np
import sys
import time
#
import tensorflow as tf
from tensorflow import keras

#--- user modules
sys.path.append(confParser['input files']['lib_path'])
import LammpsPostProcess as lp
import utility as utl
import buildDescriptors as bd
import imp
imp.reload(utl)
imp.reload(lp)
imp.reload(bd)


importing Jupyter notebook from buildDescriptors.ipynb
conf. file sections: ['flags', 'input files', 'EnergyBarrier', 'neural net', 'ml mc']


<module 'buildDescriptors' from 'buildDescriptors.ipynb'>

# MachineLeranedMC

In [31]:
class MachineLeranedMC( bd.ParseConfiguration,
                        bd.EnergyBarrier,
                      ):
    '''
    Performs Machine Learned Monte Carlo Swaps
    '''
    
    def __init__(self,
                 confParser, 
                 verbose = False
                ):
        
#         bd.ParseConfiguration.__init__(self, confParser, verbose = verbose )
        self.verbose     =  verbose 
        self.confParser  =  confParser
        
    def Parse(self,fp):
        t0 = time.time()
        self.lmpData = lp.ReadDumpFile( '%s'%(fp) ) 
        self.lmpData.GetCords( ncount = sys.maxsize)
        if self.verbose:
            print('elapsed time=%s s'%(time.time()-t0))
            print('time steps:',self.lmpData.coord_atoms_broken.keys())
            display(self.lmpData.coord_atoms_broken[0].head())

                    
    def GetDescriptors( self ):
        '''
        Compute structural descriptors 
        '''
        
        bd.EnergyBarrier.__init__( self,
                                  None,#'%s/EVENTS_DIR'%self.confParser['input files']['input_path'],
                                  None,#'%s/EVLIST_DIR'%self.confParser['input files']['input_path'],
                                  self.lmpData,
                                  None,# self.lmpDisp,
                                   verbose    = self.verbose,
                                   nconf      = 2, #--- only two events
                                   confParser = self.confParser,
                                   species    = ['Ni'], #'Ni Co Cr'.split()
                                   r_cut      = eval(self.confParser['EnergyBarrier']['r_cut']),
                                   dr         = eval(self.confParser['EnergyBarrier']['dr']),
                                   scale      = eval(self.confParser['EnergyBarrier']['scale']),
                                   n_max      = 8,
                                   l_max      = 6,

                      )
        
        self.perAtomData = self.lmpData
        self.SetDescriptors(
                      #soap = False,
                      #acsf = True,   
                      gr = True,
                     )
        
    def GetDefects( self, fp ):
        '''
        Classify Defects
        '''
        
        #--- load ml model
        model = keras.models.load_model(fp)

        #---------------
        #--- zscore X
        #---------------        
        X      = np.c_[self.descriptors ]
        scaler = StandardScaler()
        scaler.fit(X)
        X      = scaler.transform( X )

        #--- predict classes
        predict_x = model.predict( X ) 
        self.predict_classes = np.argmax( predict_x, axis=1 )
    
    
    def GetDisp( self ):
        '''
        Predict Displacements
        '''

        #--- load ml model
        model = keras.models.load_model(fp) #"best_model/convnetClassifier_from_scratch.tf")
        
        #---------------
        #--- zscore X
        #---------------        
        filtr  = self.predict_classes == 1
        X      = np.c_[self.descriptors[ filtr ] ]
        scaler = StandardScaler()
        scaler.fit(X)
        X      = scaler.transform( X )

        self.predict_disp = model.predict( X )
        
    
    def BuildCatalog( self ):
#         rate_constant_prefactor = ?
        columns = 'AtomId barrier true_rate dx dy dz'.split()


## main()

In [32]:
#--- parse conf. file
confParser = configparser.ConfigParser()
confParser.read('configuration.ini')
print('conf. file sections:',confParser.sections())



mlmc  =  MachineLeranedMC(confParser,verbose = True)
    
#--- parse atom positions
mlmc.Parse('%s/%s'%(confParser['ml mc']['input_path'],confParser['ml mc']['dump_file']))
    
#--- build descriptors
mlmc.GetDescriptors()

#--- identify defects
mlmc.GetDefects('%s/%s'%(confParser['ml mc']['input_path'],confParser['ml mc']['classifier_load']))

#--- predict diffusion paths
#mlmc.GetDisp()


#--- build catalog
#mlmc.BuildCatalog()

conf. file sections: ['flags', 'input files', 'EnergyBarrier', 'neural net', 'ml mc']
elapsed time=0.17144393920898438 s
time steps: dict_keys([0, 1, 2])


,id,type,x,y,z
0,1,1,19.823929,18.054071,19.823696
1,2,1,0.353771,16.284138,2.124210
2,3,1,18.054001,16.284338,5.664271
3,4,1,3.893790,12.744229,16.283861
4,5,1,9.204130,5.663917,0.354000


Build neighbor list: elapsed time=4.76891303062439 s


,id,type,J,Jtype,DIST,DX,DY,DZ,PBC_SHIFT_X,PBC_SHIFT_Y,PBC_SHIFT_Z
0,1,1,1270.0,1.0,2.502422,1.769480,-1.769480,-0.000641,0.0,0.0,0.0
1,1,1,900.0,1.0,2.502919,1.769733,1.769928,0.000032,0.0,0.0,0.0
2,1,1,1340.0,1.0,2.503296,0.000023,1.769928,1.770267,0.0,0.0,0.0
3,1,1,1348.0,1.0,2.502764,-0.000206,1.769928,-1.769514,0.0,0.0,0.0
4,1,1,1099.0,1.0,2.503545,1.769184,0.001183,-1.771362,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
16435,1371,1,585.0,1.0,2.503169,1.770072,-0.000028,-1.769943,0.0,0.0,0.0
16436,1371,1,1055.0,1.0,2.503169,0.000028,-1.770073,-1.769943,0.0,0.0,0.0
16437,1371,1,275.0,1.0,2.503280,1.770044,1.770129,-0.000004,0.0,0.0,0.0
16438,1371,1,1100.0,1.0,2.503281,-1.770129,-1.770044,-0.000004,0.0,0.0,0.0


xdim= 21
ydim= 21
zdim= 21
kde: elapsed time=6.760073900222778 s
rho.shape:(1371,9261)


NotFoundError: Op type not registered 'DisableCopyOnRead' in binary running on MacBook-Pro-5.local. Make sure the Op and Kernel are registered in the binary running in this process. Note that if you are loading a saved graph which used ops from tf.contrib, accessing (e.g.) `tf.contrib.resampler` should be done before importing the graph, as contrib ops are lazily registered when the module is first accessed.

In [ ]:
mlmc.descriptors

In [ ]:
dir(mlmc)